In [118]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [135]:
# turn list of lists of equivalent braids with corresponding answers into trainable data

def parse_equivalent_braids(allBraids, answers, maxCrossings):

    x_data = []
    y_data = []

    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in tf.keras.preprocessing.sequence.pad_sequences(
            equivalentBraids, padding="post", maxlen=maxCrossings):
            x_data.append(braid)
            y_data.append(1 if answer=="Y" else 0)
        
    return np.asarray(x_data), np.asarray(y_data)

In [136]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('~/Desktop/Math 498/data/knot_data.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)

In [171]:
maxStrands = 6
maxCrossings = 14

In [145]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["Q-Positive"], maxCrossings)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["Q-Positive"], maxCrossings)

In [206]:
# build model from functional api

encoder = layers.IntegerLookup(
    vocabulary=list(range(-maxStrands, maxStrands+1)),
    output_mode="one_hot",
    num_oov_indices=0)

inputs = keras.Input(shape=(14,))

x = encoder(inputs)
#x = layers.Flatten()(x)
#x = layers.Dense(64, activation="relu")(x)
#x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

ValueError: Exception encountered when calling layer "integer_lookup_19" (type IntegerLookup).

When output_mode is not `'int'`, maximum supported output rank is 2. Received output_mode one_hot and input shape (None, 14), which would result in output rank 3.

Call arguments received by layer "integer_lookup_19" (type IntegerLookup):
  • inputs=tf.Tensor(shape=(None, 14), dtype=float32)

In [207]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="quasipositivity_functional")

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 14, 13), dtype=tf.float32, name='input_22'), name='input_22', description="created by layer 'input_22'") at layer "flatten_23". The following previous layers were accessed without issue: []

In [205]:
model.summary()

Model: "quasipositivity_functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 14, 13)]          0         
                                                                 
 flatten_23 (Flatten)        (None, 182)               0         
                                                                 
 dense_60 (Dense)            (None, 64)                11712     
                                                                 
 dense_61 (Dense)            (None, 64)                4160      
                                                                 
 dense_62 (Dense)            (None, 1)                 65        
                                                                 
Total params: 15,937
Trainable params: 15,937
Non-trainable params: 0
_________________________________________________________________


In [189]:
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=5
)

Epoch 1/5


ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/keras/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

In [198]:
encoder(x_train[56])

<tf.Tensor: shape=(14, 13), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [200]:
x_train[0].shape

(14,)

In [201]:
encoder(x_train[0])

<tf.Tensor: shape=(14, 13), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [202]:
layers.Flatten()(encoder(x_train[0]))

<tf.Tensor: shape=(14, 13), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>